# LlamaParse + Cortex Search

This notebook walks through how to parse a complex report with LlamaParse, how to load it in Snowflake, and how to create a RAG via Cortex Search on top of the data loaded into Snowflake.

## Set key and import libraries
For this example, you will need a LlamaCloud API key and a Snowflake account. Get LlamaCloud API key following these [instructions](https://docs.cloud.llamaindex.ai/api_key), and signup for Snowflake [here](https://signup.snowflake.com/).

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

# llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

# snowflake
os.environ["SNOWFLAKE_ACCOUNT"] = "..." # note: "_" can cause problems with the connection, use "-" instead
os.environ["SNOWFLAKE_USER"] = "..."
os.environ["SNOWFLAKE_PASSWORD"] = "..."
os.environ["SNOWFLAKE_ROLE"] = "..."
os.environ["SNOWFLAKE_WAREHOUSE"] = "..."
os.environ["SNOWFLAKE_DATABASE"] = "SEC_10KS" # note: make sure to use a database that already
os.environ["SNOWFLAKE_SCHEMA"] = "PUBLIC"

## Load Data

Use Snowflake's latest 10K or your favorite PDF.

To use Snowflake's latest 10K, download it from [here](https://d18rn0p25nwr6d.cloudfront.net/CIK-0001640147/663fb935-b123-4bbb-8827-905bcbb8953c.pdf) and rename it as `snowflake_2025_10k.pdf`.

## Parse PDF

In [ ]:
from llama_cloud_services import LlamaParse

parser = LlamaParse(
    num_workers=4,       # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="en",       # optionally define a language, default=en
)

# sync
result = parser.parse("./snowflake_2025_10k.pdf")

In [ ]:
# get the llama-index markdown documents
markdown_documents = result.get_markdown_documents(split_by_page=True)

## Convert LlamaIndex Documents to Dataframe

In [ ]:
import pandas as pd

def documents_to_dataframe(documents):
    rows = []
    for doc in documents:
        row = {}
        # Store document ID
        row["ID"] = doc.id_
        # Add all metadata items as separate columns
        for key, value in doc.metadata.items():
            row[key] = value
        # Get text from the text_resource attribute, if available
        row["text"] = getattr(doc.text_resource, "text", None)
        rows.append(row)
    return pd.DataFrame(rows)

In [ ]:
documents_df = documents_to_dataframe(markdown_documents)

In [ ]:
documents_df

## Write DataFrame to Snowflake table

In [2]:
from snowflake.snowpark import Session

# Create Snowpark session
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),            
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WARHEOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

session = Session.builder.configs(connection_parameters).create()

/opt/anaconda3/envs/oss_rag_stack/lib/python3.11/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(
/opt/anaconda3/envs/oss_rag_stack/lib/python3.11/site-packages/snowflake/snowpark/session.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
# convert to Snowpark DataFrame
snowpark_df = session.create_dataframe(documents_df)

In [ ]:
# Write Snowpark DataFrame to a Snowflake table
# Use 'overwrite' to replace table or 'append' to add to existing table
snowpark_df.write.mode("overwrite").save_as_table("snowflake_10k")

In [ ]:
create_search_service_sql = """
CREATE OR REPLACE CORTEX SEARCH SERVICE SNOWFLAKE_10K_SEARCH_SERVICE
  ON TEXT
  ATTRIBUTES ID, PAGE_NUMBER, FILE_NAME
  WAREHOUSE = S
  TARGET_LAG = '1 hour'
    AS (
      SELECT
        ID,
        "page_number" AS PAGE_NUMBER,
        "file_name" AS FILE_NAME,
        "text" AS TEXT
      FROM SEC_10KS.PUBLIC.SNOWFLAKE_10K
      );
"""
session.sql(create_search_service_sql).collect()

In [3]:
import os
from snowflake.core import Root
from typing import List
from snowflake.snowpark.session import Session

class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 4):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(session)

        search_service = (root
          .databases["SEC_10KS"]
          .schemas["PUBLIC"]
          .cortex_search_services["SNOWFLAKE_10K_SEARCH_SERVICE"]
        )
        resp = search_service.search(
          query=query,
          columns=["text"],
          limit=self._limit_to_retrieve
        )

        if resp.results:
            return [curr["text"] for curr in resp.results]
        else:
            return []
        
retriever = CortexSearchRetriever(snowpark_session=session, limit_to_retrieve=5)

In [5]:
retrieved_context = retriever.retrieve(query="What was the product revenue for Snowflake in 2025 Q1")

retrieved_context

['# Table of Contents\n\n# Selected Financial Information\n\n| Fiscal Year Ended January 31,                      | 2025          | 2024        | 2023        |\n| -------------------------------------------------- | ------------- | ----------- | ----------- |\n| Revenue                                            | $3,626,396    | $2,806,489  | $2,065,659  |\n| Cost of revenue and operating expenses:            |               |             |             |\n| Cost of product revenue(1)(2)                      | $992,069      | $701,200    | $547,547    |\n| Cost of professional services and other revenue(2) | $222,604      | $197,358    | $169,993    |\n| Sales and marketing(2)                             | $1,672,092    | $1,391,747  | $1,106,507  |\n| Research and development(2)                        | $1,783,379    | $1,287,949  | $788,058    |\n| General and administrative(2)                      | $412,262      | $323,008    | $295,821    |\n| Interest income                      

## Create a RAG

In [15]:
from snowflake.cortex import complete

class RAG:

    def __init__(self, session):
        self.session = session
        self.retriever = CortexSearchRetriever(snowpark_session=self.session, limit_to_retrieve=10)

    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.retriever.retrieve(query)

    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""
          You are an expert assistant extracting information from context provided.
          Answer the question in long-form, fully and completely, based on the context. Do not hallucinate.
          If you don´t have the information just say so. If you do have the information you need, just tell me the answer.
          Context: {context_str}
          Question:
          {query}
          Answer:
        """
        response = ""
        response = complete("claude-4-sonnet", prompt, session = session)
        return response

    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)


rag = RAG(session)

In [26]:
response = rag.query("What was the product revenue and gross margin for Snowflake in FY 2024?")

In [27]:
from IPython.display import Markdown

display(Markdown(response))

Based on the financial information provided in the context, for Snowflake's fiscal year 2024 (ended January 31, 2024):

**Product Revenue:** $2,666,849 thousand (or approximately $2.67 billion)

**Product Gross Margin:** 74%

The context shows that product revenue represented 95% of total revenue in FY 2024, with total revenue being $2,806,489 thousand. The product gross margin of 74% indicates that Snowflake had strong profitability on its core product offerings, though this declined to 71% in FY 2025 primarily due to newly launched product capabilities and features that had not yet reached economies of scale.